# CANデータGPU処理ベンチマーク

CANバイナリデータのGPU処理とCPU処理の比較、およびParquet出力の検証を行います。

## 1. 環境設定とインポート

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path
import sys

# Add parent directory to path for imports
sys.path.append('..')

# Import decoders
#from gpu_can_decoder import GPUCANDecoder
from gpu_can_decoder_optimized import OptimizedGPUCANDecoder
from cpu_can_decoder import CPUCANDecoder

print("ライブラリのインポート完了")

ライブラリのインポート完了


## 2. ベンチマーク用データ生成

In [2]:
# テスト用データの生成 - 1,000,000メッセージ以上のみ
test_sizes = [
    1_000_000,     # 1M
    5_000_000,     # 5M
    10_000_000,    # 10M
]
print("テストデータサイズ:", test_sizes)

テストデータサイズ: [1000000, 5000000, 10000000]


In [3]:
def generate_synthetic_can_data(n_messages):
    """合成CANデータの生成（OpenPilot DBCファイルに準拠）"""
    # リアルなCANデータ分布を模倣
    address_distribution = {
        170: 0.037,  # 4輪速度
        37: 0.037,   # ステアリング
        36: 0.037,
        740: 0.044,
        608: 0.022,
        180: 0.018,
    }
    
    # アドレスを生成
    addresses = []
    for addr, prob in address_distribution.items():
        count = int(n_messages * prob)
        addresses.extend([addr] * count)
    
    # 残りはランダムなアドレス
    remaining = n_messages - len(addresses)
    other_addresses = np.random.choice([452, 466, 467, 705, 321, 562], remaining)
    addresses.extend(other_addresses)
    
    # シャッフル
    np.random.shuffle(addresses)
    addresses = np.array(addresses[:n_messages], dtype=np.int64)
    
    # タイムスタンプ（実データと同じ範囲）
    timestamps = np.linspace(46408.0, 46468.0, n_messages)
    
    # データバイト
    data_bytes = np.zeros((n_messages, 8), dtype=np.uint8)
    
    for i in range(n_messages):
        if addresses[i] == 170:  # 4輪速度
            # OpenPilot DBC: (0.01,-67.67) \"kph\" for Toyota RAV4
            for j in range(4):
                speed_kmh = np.random.uniform(55, 65)  # 55-65 km/h
                raw_value = int((speed_kmh + 67.67) / 0.01)
                data_bytes[i, j*2] = (raw_value >> 8) & 0xFF
                data_bytes[i, j*2 + 1] = raw_value & 0xFF
        elif addresses[i] == 37:  # ステアリング
            # 固定値パターン（実データと同じ）
            data_bytes[i] = [0x00, 0x00, 0x10, 0x00, 0xC0, 0x00, 0x00, 0xFD]
        else:
            # その他はランダム
            data_bytes[i] = np.random.randint(0, 256, 8, dtype=np.uint8)
    
    return timestamps, addresses, data_bytes

In [4]:
# デコーダーの初期化とウォームアップ
print("デコーダーを初期化中...")
optimized_gpu_decoder = OptimizedGPUCANDecoder(batch_size=500_000, chunk_size=1)
cpu_decoder = CPUCANDecoder(batch_size=100_000)

# CUDAカーネルのウォームアップ（初回コンパイルを事前に実行）
print("CUDAカーネルのウォームアップ中...")
warmup_data = generate_synthetic_can_data(10_000)
for i in range(2):
    print(f"  ウォームアップ {i+1}/2...")
    _ = optimized_gpu_decoder.decode_batch_for_benchmark(*warmup_data)
print("ウォームアップ完了！")

デコーダーを初期化中...
CUDAカーネルのウォームアップ中...
  ウォームアップ 1/2...


/home/ubuntu/miniconda3/envs/cudf_dev/lib/python3.12/site-packages/numba_cuda/numba/cuda/dispatcher.py:605: NumbaPerformanceWarning: Grid size 3 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


  === GPU処理の詳細 ===
  データ抽出: 0.0028秒
  物理値変換: 0.3616秒
  総処理時間: 0.3644秒
  データサイズ削減: 0.2 MB → 0.0 MB (7.4%)
  ウォームアップ 2/2...
  === GPU処理の詳細 ===
  データ抽出: 0.0007秒
  物理値変換: 0.0930秒
  総処理時間: 0.0936秒
  データサイズ削減: 0.2 MB → 0.0 MB (7.4%)
ウォームアップ完了！


## 3. ベンチマーク実行

In [ ]:
# ベンチマーク結果格納
benchmark_results = []

for n_messages in test_sizes:
    print(f"\n--- {n_messages:,} メッセージの処理 ---")
    
    # データ生成
    timestamps, addresses, data_bytes = generate_synthetic_can_data(n_messages)
    data_size_mb = (timestamps.nbytes + addresses.nbytes + data_bytes.nbytes) / (1024**2)
    
    # 抽出対象データサイズの計算（アドレス170と37のみ）
    target_mask = (addresses == 170) | (addresses == 37)
    target_count = np.sum(target_mask)
    target_size_mb = (timestamps[target_mask].nbytes + addresses[target_mask].nbytes + data_bytes[target_mask].nbytes) / (1024**2)
    
    print(f"全データサイズ: {data_size_mb:.1f} MB（内抽出対象データサイズ: {target_size_mb:.1f} MB）")
    
    # GPU処理（最適化）
    opt_gpu_start = time.time()
    opt_gpu_chunk_results = optimized_gpu_decoder.decode_batch_for_benchmark(timestamps, addresses, data_bytes)
    opt_gpu_time = time.time() - opt_gpu_start
    
    # CPU処理（全てのサイズで実測）
    cpu_start = time.time()
    cpu_results = cpu_decoder.decode_batch(timestamps, addresses, data_bytes, debug_timing=True)
    cpu_time = time.time() - cpu_start
    
    # CPU処理の詳細時間を表示（3つの主要ステップに集約）
    if '_timing' in cpu_results:
        timing = cpu_results['_timing']
        # マスク作成
        mask_time = timing.get('mask_creation', 0)
        # データ抽出（インデックス検索と配列割り当て）
        data_extract_time = timing.get('index_search', 0) + timing.get('array_allocation', 0)
        # 物理値変換（デコードループ、DataFrame作成、ソート）
        physical_convert_time = (timing.get('decode_loop', 0) + 
                               timing.get('wheel_df_creation', 0) + 
                               timing.get('wheel_sort', 0))
        
        print(f"\n  === CPU処理の詳細 ===")
        print(f"  マスク作成: {mask_time:.4f}秒")
        print(f"  データ抽出: {data_extract_time:.4f}秒")
        print(f"  物理値変換: {physical_convert_time:.4f}秒")
        print(f"  総処理時間: {cpu_time:.4f}秒")
    
    # 結果記録
    result = {
        'n_messages': n_messages,
        'data_size_mb': data_size_mb,
        'opt_gpu_time': opt_gpu_time,
        'cpu_time': cpu_time,
        'speedup': cpu_time / opt_gpu_time,
        'opt_gpu_throughput': n_messages / opt_gpu_time / 1e6,
        'cpu_throughput': n_messages / cpu_time / 1e6
    }
    benchmark_results.append(result)
    
    print(f"\nGPU処理時間: {opt_gpu_time:.4f}秒 ({result['opt_gpu_throughput']:.1f} Mmsg/s)")
    print(f"CPU処理時間: {cpu_time:.4f}秒 ({result['cpu_throughput']:.1f} Mmsg/s)")
    print(f"高速化率: {result['speedup']:.1f}x")

# DataFrameに変換
benchmark_df = pd.DataFrame(benchmark_results)
benchmark_df


--- 1,000,000 メッセージの処理 ---
全データサイズ: 22.9 MB（内抽出対象データサイズ: 1.7 MB）
  === GPU処理の詳細 ===
  データ抽出: 0.0051秒
  物理値変換: 0.0131秒
  総処理時間: 0.0182秒
  データサイズ削減: 22.9 MB → 1.7 MB (7.4%)

  === CPU処理の詳細 ===
  マスク作成: 0.0005秒
  データ抽出: 0.0009秒
  物理値変換: 0.0709秒
  総処理時間: 0.0924秒

GPU処理時間: 0.0206秒 (48.5 Mmsg/s)
CPU処理時間: 0.0924秒 (10.8 Mmsg/s)
高速化率: 4.5x

--- 5,000,000 メッセージの処理 ---
全データサイズ: 114.4 MB（内抽出対象データサイズ: 8.5 MB）
  === GPU処理の詳細 ===
  データ抽出: 0.0299秒
  物理値変換: 0.0144秒
  総処理時間: 0.0443秒
  データサイズ削減: 114.4 MB → 8.5 MB (7.4%)

  === CPU処理の詳細 ===
  マスク作成: 0.0026秒
  データ抽出: 0.0050秒
  物理値変換: 0.3606秒
  総処理時間: 0.4638秒

GPU処理時間: 0.0555秒 (90.1 Mmsg/s)
CPU処理時間: 0.4638秒 (10.8 Mmsg/s)
高速化率: 8.4x

--- 10,000,000 メッセージの処理 ---
全データサイズ: 228.9 MB（内抽出対象データサイズ: 16.9 MB）
  === GPU処理の詳細 ===
  データ抽出: 0.0484秒
  物理値変換: 0.0180秒
  総処理時間: 0.0664秒
  データサイズ削減: 228.9 MB → 16.9 MB (7.4%)
